In [27]:
class vec2i(object):
    def __init__(self, x, y):
        self.x = int(round(x))
        self.y = int(round(y))

    @classmethod
    def from_index(cls, ind):
        ssplit = ind[1:].split('P')
        yoff = 1
        if(len(ssplit) == 1):
            ssplit = ind[1:].split('N')
            yoff = -1
        
        xoff = 1
        if(ind[0] == 'N'):
            xoff = -1
        
        return cls(xoff * int(ssplit[0]), yoff * int(ssplit[1]))
        
    def index(self):
        xpart = ""
        if(self.x < 0):
            xpart = "N%i" % abs(self.x) 
        else:
            xpart = "P%i" % abs(self.x)
        ypart = ""
        if(self.y < 0):
            ypart = "N%i" % abs(self.y) 
        else:
            ypart = "P%i" % abs(self.y)
        return "%s%s" % (xpart,ypart)
    
    def step(self, v2):
        return vec2i(self.x + v2.x, self.y + v2.y)
    
    def diff(self, v2):
        return vec2i(v2.x - self.x, v2.y - self.y)
    
    def copy(self):
        return vec2i(self.x, self.y)
    
    def __str__(self):
        return ("(%i,%i)" % (self.x, self.y))
    
    def __repr__(self):
        return ("(%i,%i)" % (self.x, self.y))
    
class Area(object):
    
    def __init__(self, data):
        self.space = {}
        self.move_steps = {1:vec2i(0,-1), 2:vec2i(0,1), 3:vec2i(-1,0), 4:vec2i(1,0)}
        self.cellindices = []
        self.adjacents = {}
        
        y = 0
        for row in data:
            x = 0
            for cell in row:
                pos = vec2i(x,y)
                key = pos.index()
                self.set_alive(key, cell == '#')
                self.cellindices.append(key)
                self.adjacents[key] = []                
                for i in range(1,5):
                    adjacent = pos.step(self.move_steps[i])
                    if(adjacent.x >= 0 and adjacent.x <= 5 and
                       adjacent.y >= 0 and adjacent.y <= 5):
                        self.adjacents[key].append(adjacent.index())

                x = x + 1
            y = y + 1
    
    def is_alive(self, key):
        if(key in self.space):
            return self.space[key]
        return False
    
    def set_alive(self, key, alive):
        self.space[key] = alive
    
    def adjacent_count(self, key):
        count = 0
        for adjacent in self.adjacents[key]:
            if(self.is_alive(adjacent)):
                count = count + 1
        return count

    
    def step(self):
        count = {}
        for cell in self.cellindices:
            count[cell] = self.adjacent_count(cell)            
            
        for cell in self.cellindices:
            if(self.is_alive(cell)):
                if(count[cell] != 1):
                    self.set_alive(cell, False)
            else:
                if(count[cell] == 1 or count[cell] == 2):
                    self.set_alive(cell, True)
    
    def rating(self):
        i = 0
        rating = 0
        for cell in self.cellindices:
            if(self.is_alive(cell)):
                rating = rating + 2 ** i
            i = i + 1
            
        return rating
    
    def render(self):
        s = ''
        for y in range(5):
            for x in range(5):
                if(self.is_alive(vec2i(x,y).index())):
                    s = s + '#'
                else:
                    s = s + '.'
            s = s + '\n'
        print(s)
    
    def part1(self):
        seen = set()
        
        while(True):
            rating = self.rating()
            if(rating in seen):
                print ("part 1: rating = %i" % rating)
                break
            self.step()
            seen.add(rating)


In [28]:
test1 = ['....#','#..#.','#..##','..#..','#....']
area = Area(test1)
area.render()

....#
#..#.
#..##
..#..
#....



In [29]:
area.step()
area.render()

#..#.
####.
###.#
##.##
.##..



In [30]:
test2 = ['.....','.....','.....','#....','.#...']
area2 = Area(test2)
print(area2.rating())

2129920


In [31]:
data = ['.###.','..#.#','...##','#.###','..#..']
part1 = Area(data)
part1.part1()

part 1: rating = 27562081


In [60]:
class Cell(object):
    def __init__(self, alive):
        self.alive = alive
        self.adjacentcount = 0
        self.adjacents = []
    
    def add_adjacents(self, adjacents):
        self.adjacents.extend(adjacents)

    def count(self):
        self.adjacentcount = 0
        for adjacent in self.adjacents:
            if(adjacent.alive):
                self.adjacentcount = self.adjacentcount + 1
                
    def step(self):
        if(self.alive):
            if(self.adjacentcount != 1):
                self.alive = False
        elif(self.adjacentcount == 1 or self.adjacentcount == 2):
            self.alive = True

class RecursiveLevel(object):
    def __init__(self, data=None):
        
        self.grid = []
        self.cells = []
        
        if(data == None):
            data = []
            for y in range(5):
                data.append('.....')
        
        for y in range(5):
            self.grid.append([])
            for x in range(5):
                if(x != 2 or y != 2):
                    cell = Cell(data[y][x] == '#')
                    self.grid[y].append(cell)
                    self.cells.append(cell)
                else:
                    self.grid[y].append(None) #recursive
                    
        for y in range(5):
            for x in range(5):
                if(y == 2 and x == 2):
                    continue
                
                adjacents = []
                up = self.get_cell(x, y - 1)
                if(up != None):
                    adjacents.append(up)
                down = self.get_cell(x, y + 1)
                if(down != None):
                    adjacents.append(down)
                left = self.get_cell(x - 1, y)
                if(left != None):
                    adjacents.append(left)
                right = self.get_cell(x + 1, y)
                if(right != None):
                    adjacents.append(right)
                self.grid[y][x].add_adjacents(adjacents)
                    
        
    def get_cell(self, x, y):
        if(x < 0 or x > 4 or y < 0 or y > 4):
            return None
        return self.grid[y][x]
    
    def connect_up(self, level): #level being passed in is "up", ie self is the middle cell of level
        
        #top row connects to level.cell(2,1)
        #bottom row connects to level.cell(2,3)
        above = level.get_cell(2,1)
        below = level.get_cell(2,3)
        for x in range(5):
            self.grid[0][x].add_adjacents([above])
            self.grid[4][x].add_adjacents([below])
        above.add_adjacents(self.grid[0])
        below.add_adjacents(self.grid[4])

        #left column connects to level.cell(1,2)
        #right column connects to level.cell(3,2)
        left = level.get_cell(1,2)
        right = level.get_cell(3,2)
        lcol = []
        rcol = []
        for y in range(5):
            lcol.append(self.grid[y][0])
            self.grid[y][0].add_adjacents([left])
            rcol.append(self.grid[y][4])
            self.grid[y][4].add_adjacents([right])
        left.add_adjacents(lcol)
        right.add_adjacents(rcol)
        
    def count(self):
        for cell in self.cells:
            cell.count()
            
    def step(self):
        for cell in self.cells:
            cell.step()
            
    def total_living(self):
        living = 0
        for cell in self.cells:
            if(cell.alive):
                living = living + 1
        return living
    
    def render(self):
        s = ''
        for y in range(5):
            for x in range(5):
                if(y == 2 and x == 2):
                    s = s + '?'
                elif(self.grid[y][x].alive):
                    s = s + '#'
                else:
                    s = s + '.'
            s = s + '\n'
        print(s)
            
class RecursiveArea(object):
    
    def __init__(self, data0, levels):
        
        self.levels = []
        prior = None
        newlevel = None
        self.levelcount = levels
        for i in range(-levels, levels+1):
            if(i == 0):
                newlevel = RecursiveLevel(data0)
            else:
                newlevel = RecursiveLevel()
            self.levels.append(newlevel)
            
            if(prior != None):
                newlevel.connect_up(prior)
            prior = self.levels[-1]

        
    def step(self):
        
        for level in self.levels:
            level.count()
            
        for level in self.levels:
            level.step()
            
                
    def total_living(self):
        living = 0
        for level in self.levels:
            living = living + level.total_living()
        return living
    
    
    def render(self):
        i = -self.levelcount
        
        for level in self.levels:
            print("level %i" % i)
            level.render()
            i = i + 1
        
        

In [74]:
area = RecursiveArea(test1, 5)
for i in range(10):
    area.step()
area.render()
print(area.total_living())

level -5
..#..
.#.#.
..?.#
.#.#.
..#..

level -4
...#.
...##
..?..
...##
...#.

level -3
#.#..
.#...
..?..
.#...
#.#..

level -2
.#.##
....#
..?.#
...##
.###.

level -1
#..##
...##
..?..
...#.
.####

level 0
.#...
.#.##
.#?..
.....
.....

level 1
.##..
#..##
..?.#
##.##
#####

level 2
###..
##.#.
#.?..
.#.##
#.#..

level 3
..###
.....
#.?..
#....
#...#

level 4
.###.
#..#.
#.?..
##.#.
.....

level 5
####.
#..#.
#.?#.
####.
.....

99


In [76]:
area = RecursiveArea(data, 105)
for i in range(200):
    area.step()
print("part 2 answer: %i" % area.total_living())

part 2 answer: 1893
